In [171]:
import pandas as pd
import numpy as np

In [172]:
data = pd.read_csv('/Users/logno/Documents/GitHub/diabetes-project/diabetes.csv')
df = data.copy()

# Broad overview of the data

In [173]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [174]:
df.shape

(768, 9)

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [176]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [177]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


# Cleaning

Despite there being no N/A values in this dataset, I believe that there are some values in certain columns that are incorrect. For example, it is impossible for a person to have an insulin value of zero, and if they did, that would be an issue. In this section, I will clean up these values and ensure that all of the data makes sense.

## Finding which columns to check

In [178]:
df.loc[df['Glucose'] == 0].shape

(5, 9)

In [179]:
df.loc[df['BloodPressure'] == 0].shape

(35, 9)

In [180]:
df.loc[df['BMI'] == 0].shape

(11, 9)

In [181]:
df.loc[df['Age'] == 0].shape

(0, 9)

In [182]:
df.loc[df['DiabetesPedigreeFunction'] == 0].shape

(0, 9)

It seems that the columns with the missing values are glucose, bloodpressure, and BMI. 

## Glucose

In [183]:
# Finding the rows that have a glucose of 0
df.loc[df['Glucose'] == 0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


Since there are only 5 rows, I think that it would be best to drop them.

In [184]:
# Dropping the problem rows and creating a new dataframe
df = df.drop(df.index[[75,182,342,349,502]])

In [185]:
# Checking if the values were dropped
df.loc[df['Glucose'] == 0].shape

(0, 9)

## Blood Pressure

In [186]:
df['BloodPressure'].describe()

count    763.000000
mean      69.115334
std       19.399240
min        0.000000
25%       62.000000
50%       72.000000
75%       80.000000
max      122.000000
Name: BloodPressure, dtype: float64

In [187]:
df.loc[df['BloodPressure'] == 0].shape

(35, 9)

In [188]:
# Finding how many null values there are
df['BloodPressure'] = df['BloodPressure'].replace({0: np.nan})
df['BloodPressure'].isnull().sum()

35

Since there are a fair number of null values, I will impute them using the median.

In [189]:
# Imputing values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')

df["BloodPressure"] = imp.fit_transform(df[["BloodPressure"]]).ravel()
df['BloodPressure'].isnull().sum()

0

## BMI

In [190]:
df.loc[df['BMI'] == 0].shape

(11, 9)

In [191]:
df['BMI'].describe()

count    763.000000
mean      31.986763
std        7.893221
min        0.000000
25%       27.300000
50%       32.000000
75%       36.550000
max       67.100000
Name: BMI, dtype: float64

I will impute these values as well, just for fun. I can come back to this later.

In [192]:
# Imputing values
df["BMI"] = imp.fit_transform(df[["BMI"]]).ravel()
df['BMI'].isnull().sum()

0

Time to double check that I am done cleaning.

In [193]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [194]:
df.shape

(763, 9)